# https://www.youtube.com/watch?v=wbfXaqjIrJ0

In [33]:
import pandas as pd
import numpy as np
import os 
!pip install plotly
import plotly
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.io as pio
from datetime import datetime
import scipy as sp
import time
from random import random
import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta
import time 
import os
def create_data_and_save(instrument,start,end,pair,hours = 3,granularity = "M1"):

    print('CREATING DATASET...')
    s = time.time()
    print('PARAMS:',start,end,pair,granularity)
    cur = start

    #i = 'GBP_JPY'
    total_df = 0
    dct = {}
    
    dir_name = os.getcwd() + '/' + pair + '_' + granularity
    
    isdir = os.path.isdir(dir_name) 
    if isdir:
        pass
    else:
        print('MAKING DIRECTORY:',dir_name)
        os.mkdir(dir_name) 
    path = dir_name + '/' + str(date(start.year,start.month,start.day)) + '_' + str(date(end.year,end.month,end.day)) + '.csv'
    print('PATH:',path)
    while cur < end:
        print(cur)    
        df = convert_to_simple_df(instrument = instrument,granularity = granularity,count = 5000,start = cur,end = cur + timedelta(hours = hours))
        

        cur = cur + timedelta(hours = hours)

        print(df.shape)
        if df.shape[1] == 1:
            pass
        else:
            try:
                total_df = total_df.append(df)
            except:
                total_df = df
                
            dct[str(cur)] = df

    print(total_df.shape)
    total_df.to_csv(path,index = False)
    e = time.time()
    print('TOTAL DATASET CONSTRUCTION TIME:',(e-s)/60,' MINUTES')
    
    
def smma(df,period = 14):
    print('GETTING SMMA INDICATOR FOR:',period)
    close_col = df.columns.get_loc('c')
    df['smma_' + str(period)] = df['c']
    smma_col = df.columns.get_loc('smma_' + str(period))
    arr = df.values
    
    #SMMA CALC:
    
    #SUM1=SUM (CLOSE, N)

    #SMMA1 = SUM1/ N

    #The second and subsequent moving averages are calculated according to this formula:

    #SMMA (i) = (SUM1 – SMMA1+CLOSE (i))/ N    
    
   # Where:

    #SUM1 – is the total sum of closing prices for N periods;
    #SMMA1 – is the smoothed moving average of the first bar;
    #SMMA (i) – is the smoothed moving average of the current bar (except the first one);
    #CLOSE (i) – is the current closing price;
    #N – is the smoothing period.    

    for i in range(period,arr.shape[0]):
        if i == period:
            sum1 = sum(arr[:i,close_col])

            smma1 = sum1 / period
 
            arr[i,smma_col] = smma1
        elif i == period + 1:
            arr[i,smma_col] = (smma1 * (period - 1) + arr[i,close_col]) / period
            
        else:

            prev_sum = arr[i - 1,smma_col] * period
            arr[i,smma_col] = (prev_sum - arr[i - 1,smma_col] + arr[i,close_col]) / period
        
        
    return pd.DataFrame(arr,columns = df.columns)

def ema(df,num = 14):
    print('GETTING EMA INDICATOR FOR:',num)
    close_col = df.columns.get_loc('c')
    df['ema_' + str(num)] = df['c']
    arr = df.values
    mult = 2/ (num + 1)
    for i in range(num,arr.shape[0]):
        sma = sum(arr[i - num + 1: i + 1,close_col]) / num
        arr[i,-1] = ((arr[i,close_col] - arr[i - 1,-1]) * mult) + arr[i - 1,-1]
    return pd.DataFrame(arr,columns = df.columns)

def engulfing_candle(df):
    print('GETTING ENGULFING CANDLES...')
    df['bearish_engulfing'] = 0
    df['bullish_engulfing'] = 0
    col1 = df.columns.get_loc('bearish_engulfing')
    col2 = df.columns.get_loc('bullish_engulfing')
    
    open_col = df.columns.get_loc('o')
    close_col = df.columns.get_loc('c')
    arr = df.values 
    for i in range(arr.shape[0]):
        obp = arr[i-1,open_col] #open[1]
        cbp = arr[i-1,close_col] #close[1]
        obc = arr[i,open_col] #open
        cbc = arr[i,close_col] #close        
        #If current bar open is less than equal to the previous bar close AND current bar open is less than previous bar open AND current bar close is greater than previous bar open THEN True
        if obc <= cbp and obc < obp and cbc > obp:
            arr[i,col2] = 1
        
        #If current bar open is greater than equal to previous bar close AND current bar open is greater than previous bar open AND current bar close is less than previous bar open THEN True
        elif obc >= cbp and obc > obp and cbc < obp:    
            arr[i,col1] = 1
        
    return pd.DataFrame(arr,columns = df.columns)


def three_line_strike(df):
    print('GETTING 3 LINE STRIKE...')
    df['bearish_tls'] = 0
    df['bullish_tls'] = 0
    col1 = df.columns.get_loc('bearish_tls')
    col2 = df.columns.get_loc('bullish_tls')
    open_col = df.columns.get_loc('o')
    close_col = df.columns.get_loc('c')
    arr = df.values 
    for i in range(4,arr.shape[0]):    
        if arr[i - 3,close_col] > arr[i - 3,open_col] and arr[i - 2,close_col] > arr[i - 2,open_col] and arr[i - 1,close_col] > arr[i - 1,open_col] and arr[i,close_col] < arr[i - 1,open_col]: 
            arr[i,col1] = 1
        if arr[i - 3,close_col] < arr[i - 3,open_col] and arr[i - 2,close_col] < arr[i - 2,open_col] and arr[i - 1,close_col] < arr[i - 1,open_col] and arr[i,close_col] > arr[i - 1,open_col]: 
            arr[i,col2] = 1 
            
    return pd.DataFrame(arr,columns = df.columns)

def convert_timestamp(df):
    time_col = df.columns.get_loc('time')
    arr = df.values 
    for i in range(arr.shape[0]):
        arr[i,time_col] = datetime.strptime(arr[i,time_col][:-4], '%Y-%m-%dT%H:%M:%S.%f')
    return pd.DataFrame(arr,columns = df.columns)

def get_trading_session(df,london_start = 7,london_end = 16,ny_start = 13,ny_end = 22):
    print('GETTING TRADING SESSIONS')
    print('LONDON TRADING SESSION UTC',london_start,london_end)
    print('NEW YORK TRADING SESSION UTC',ny_start,ny_end)
    df['new_york'] = 0
    df['london'] = 0
    
    
    col1 = df.columns.get_loc('new_york')
    col2 = df.columns.get_loc('london')
    utc = df.columns.get_loc('time')
    arr = df.values 

    for i in range(arr.shape[0]):
        if arr[i,utc].hour >= london_start and arr[i,utc].hour < london_end:
            arr[i,col2] = 1
        if arr[i,utc].hour >= ny_start and arr[i,utc].hour < ny_end:
            arr[i,col1] = 1     
    return pd.DataFrame(arr,columns = df.columns)
def clean_and_save_data(pair = 'EUR_USD'
                    ,granularity = 'M5'
                    ,start = datetime(2016,1,1,0,0,0)
                    ,end = datetime(2022,7,31,0,0,0)):
    dir_name = os.getcwd() + '/' + pair + '_' + granularity
    path = dir_name + '/' + str(date(start.year,start.month,start.day)) + '_' + str(date(end.year,end.month,end.day)) + '.csv'
    print('PATH:',path)
    df = pd.read_csv(path)
    print(df.shape)
    df = smma(df,period = 21)
    df = smma(df,period = 50)
    df = smma(df,period = 200)

    df = ema(df,num = 14 * 3)
    df = ema(df,num = 50 * 3)
    df = ema(df,num = 200 * 3)
    df = engulfing_candle(df)
    df = three_line_strike(df)



    df['o'] = df['o'].astype(float)
    df['h'] = df['h'].astype(float)
    df['l'] = df['l'].astype(float)
    df['c'] = df['c'].astype(float)

    df = convert_timestamp(df)
    df = get_trading_session(df,london_start = 7,london_end = 16,ny_start = 13,ny_end = 22)
    print('SAVING DATA AS:',path)
    df.to_csv(path,index = False)
    print()
jpy_pairs=['GBP_JPY','USD_JPY','EUR_JPY','AUD_JPY','CAD_JPY','CHF_JPY']
major_pairs=['GBP_USD','USD_CAD','USD_CHF','EUR_USD','AUD_USD','NZD_USD']
cross_pairs=['AUD_CAD','AUD_CHF','AUD_NZD','CAD_CHF','EUR_AUD','EUR_CAD','EUR_CHF','NZD_CAD','NZD_CHF']
all_pairs=['XAU_USD','GBP_JPY','USD_JPY','EUR_JPY','AUD_JPY','CAD_JPY','CHF_JPY',
           'GBP_USD','USD_CAD','USD_CHF','EUR_USD','AUD_USD',
           'NZD_USD','AUD_CAD','AUD_CHF','AUD_NZD','CAD_CHF',
           'EUR_AUD','EUR_CAD','EUR_CHF','NZD_CAD','NZD_CHF'] 


all_pairs=['GBP_JPY','EUR_USD','USD_JPY','EUR_JPY','AUD_JPY','CAD_JPY','CHF_JPY',
           'GBP_USD','USD_CAD','USD_CHF','AUD_USD',
           'NZD_USD','AUD_CAD','AUD_CHF','AUD_NZD','CAD_CHF',
           'EUR_AUD','EUR_CAD','EUR_CHF','NZD_CAD','NZD_CHF'] 

import os


pair = 'EUR_USD'
granularity = 'M5'
start = datetime(2016,1,1,0,0,0)
end = datetime(2022,7,31,0,0,0)


for pair in all_pairs:
    print(pair)
    clean_and_save_data(pair = pair,granularity = granularity,start = start,end = end)
    
    
    
    


[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: pip install --upgrade pip
GBP_JPY
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/GBP_JPY_M5/2016-01-01_2022-07-31.csv
(491385, 8)
GETTING SMMA INDICATOR FOR: 21
GETTING SMMA INDICATOR FOR: 50
GETTING SMMA INDICATOR FOR: 200
GETTING EMA INDICATOR FOR: 42
GETTING EMA INDICATOR FOR: 150
GETTING EMA INDICATOR FOR: 600
GETTING ENGULFING CANDLES...
GETTING 3 LINE STRIKE...
GETTING TRADING SESSIONS
LONDON TRADING SESSION UTC 7 16
NEW YORK TRADING SESSION UTC 13 22
SAVING DATA AS: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/GBP_JPY_M5/2016-01-01_2022-07-31.csv

EUR_USD
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/EUR_USD_M5/2016-01-01_2022-07-31.csv
(490023, 8)
GETTING SMMA INDICATOR FOR: 21
GETTING SMMA INDICATOR FOR: 50
GETTING SMMA INDICATOR FOR: 200
GETTING EMA INDICATOR FOR: 42
GETTING EMA INDICATOR FOR: 150
GETTING EMA INDICATOR FOR: 600
GETTING ENG

In [ ]:
def clean_and_save_data():
    dir_name = os.getcwd() + '/' + pair + '_' + granularity
    path = dir_name + '/' + str(date(start.year,start.month,start.day)) + '_' + str(date(end.year,end.month,end.day)) + '.csv'
    print('PATH:',path)
    df = pd.read_csv(path)
    print(df.shape)
    df = smma(df,period = 21)
    df = smma(df,period = 50)
    df = smma(df,period = 200)

    df = ema(df,num = 14 * 3)
    df = ema(df,num = 50 * 3)
    df = ema(df,num = 200 * 3)
    df = engulfing_candle(df)
    df = three_line_strike(df)



    df['o'] = df['o'].astype(float)
    df['h'] = df['h'].astype(float)
    df['l'] = df['l'].astype(float)
    df['c'] = df['c'].astype(float)

    df = convert_timestamp(df)
    df = get_trading_session(df,london_start = 7,london_end = 16,ny_start = 13,ny_end = 22)
    print('SAVING DATA AS:',path)
    df.to_csv(path,index = False)
    print()

In [26]:
df.head()

,time,volume,o,h,l,c,utc_timestamp,est_timestamp,smma_21,smma_50,smma_200,ema_42,ema_150,ema_600,bearish_engulfing,bullish_engulfing,bearish_tls,bullish_tls,new_york,london
0,2016-01-03T22:00:00.000000000Z,6,1.08743,1.08746,1.08724,1.08726,2016-01-03 22:00:00+00:00,2016-01-03 17:00:00-05:00,1.08726,1.08726,1.08726,1.08726,1.08726,1.08726,0,0,0,0,0,0
1,2016-01-03T22:05:00.000000000Z,53,1.08735,1.08738,1.0871,1.0871,2016-01-03 22:05:00+00:00,2016-01-03 17:05:00-05:00,1.0871,1.0871,1.0871,1.0871,1.0871,1.0871,0,0,0,0,0,0
2,2016-01-03T22:10:00.000000000Z,21,1.08715,1.08715,1.08674,1.08681,2016-01-03 22:10:00+00:00,2016-01-03 17:10:00-05:00,1.08681,1.08681,1.08681,1.08681,1.08681,1.08681,0,0,0,0,0,0
3,2016-01-03T22:15:00.000000000Z,19,1.08671,1.08694,1.08669,1.0868,2016-01-03 22:15:00+00:00,2016-01-03 17:15:00-05:00,1.0868,1.0868,1.0868,1.0868,1.0868,1.0868,0,0,0,0,0,0
4,2016-01-03T22:20:00.000000000Z,47,1.08684,1.08717,1.0867,1.0867,2016-01-03 22:20:00+00:00,2016-01-03 17:20:00-05:00,1.0867,1.0867,1.0867,1.0867,1.0867,1.0867,1,0,0,0,0,0


In [29]:
df['utc_timestamp'].iloc[0]

'2016-01-03 22:00:00+00:00'

In [ ]:
def get_trading_session(df,london_start = 7,london_end = 16,ny_start = 13,ny_end = 22):
    print('GETTING TRADING SESSIONS')
    print('LONDON TRADING SESSION UTC',london_start,london_end)
    print('NEW YORK TRADING SESSION UTC',ny_start,ny_end)
    df['new_york'] = 0
    df['london'] = 0
    
    
    col1 = df.columns.get_loc('new_york')
    col2 = df.columns.get_loc('london')
    utc = df.columns.get_loc('utc_timestamp')
    est = df.columns.get_loc('est_timestamp')
    arr = df.values 

    for i in range(arr.shape[0]):
        if arr[i,utc] >= london_start and arr[i,utc] < london_end:
            arr[i,col2] = 1
        if arr[i,utc] >= ny_start and arr[i,utc] < ny_end:
            arr[i,col1] = 1        

In [21]:
start.hour

0

In [22]:
end.hour

0

In [ ]:


def engulfing_candle(df):
    print('GETTING ENGULFING CANDLES...')
    df['bearish_engulfing'] = 0
    df['bullish_engulfing'] = 0
    col1 = df.columns.get_loc('bearish_engulfing')
    col2 = df.columns.get_loc('bullish_engulfing')
    
    open_col = df.columns.get_loc('o')
    close_col = df.columns.get_loc('c')
    arr = df.values 
    for i in range(arr.shape[0]):
        obp = arr[i-1,open_col] #open[1]
        cbp = arr[i-1,close_col] #close[1]
        obc = arr[i,open_col] #open
        cbc = arr[i,close_col] #close        
        #If current bar open is less than equal to the previous bar close AND current bar open is less than previous bar open AND current bar close is greater than previous bar open THEN True
        if obc <= cbp and obc < obp and cbc > obp:
            arr[i,col2] = 1
        
        #If current bar open is greater than equal to previous bar close AND current bar open is greater than previous bar open AND current bar close is less than previous bar open THEN True
        elif obc >= cbp and obc > obp and cbc < obp:    
            arr[i,col] = 1
        
        

In [ ]:
def three_line_strike(df):
    print('GETTING 3 LINE STRIKE...')
    df['bearish_tls'] = 0
    df['bullish_tls'] = 0
    col1 = df.columns.get_loc('bearish_tls')
    col2 = df.columns.get_loc('bullish_tls')
    open_col = df.columns.get_loc('o')
    close_col = df.columns.get_loc('c')
    arr = df.values 
    for i in range(4,arr.shape[0]):    
        if arr[i - 3,close_col] > arr[i - 3,open_col] and arr[i - 2,close_col] > arr[i - 2,open_col] and arr[i - 1,close_col] > arr[i - 1,open_col] and arr[i,close_col] < arr[i - 1,open_col]: 
            arr[i,col1] = 1
        if arr[i - 3,close_col] < arr[i - 3,open_col] and arr[i - 2,close_col] < arr[i - 2,open_col] and arr[i - 1,close_col] < arr[i - 1,open_col] and arr[i,close_col] > arr[i - 1,open_col]: 
            arr[i,col2] = 1   
            
            
        
        
        
        

In [ ]:
import pandas as pd
import numpy as np
import os 
!pip install plotly
import plotly
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.io as pio
from datetime import datetime
import scipy as sp
import time
from random import random
pio.renderers.default = "iframe"
def create_spike_trigger(df,col = 'delta_max',lookback_threshold = 3600):
    print('CREATING SPIKE TRIGGER...')
    df[col + '_spike_trigger'] = 0
    col_loc = df.columns.get_loc(col)
    arr = df.values
    for i in range(lookback_threshold,arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        try:
            cur_max = max(arr[i - lookback_threshold : i ,col_loc])
        except:
            print('ERROR',i)
            cur_max = 100
        if arr[i,col_loc] > cur_max and sum(arr[i - 100 : i,-1]) == 0:
            arr[i,-1] = 1
    return pd.DataFrame(arr,columns = df.columns)

def create_wick_trigger(df,col = 'wick',lookback_threshold = 3600):
    print('CREATING WICK TRIGGER...')
    df['wick'] = 0
    df.loc[df[df['delta_vector'] < 0].index,'wick'] = df.loc[df[df['delta_vector'] < 0].index,'c'] - df.loc[df[df['delta_vector'] < 0].index,'l']
    df.loc[df[df['delta_vector'] > 0].index,'wick'] = df.loc[df[df['delta_vector'] > 0].index,'h'] - df.loc[df[df['delta_vector'] > 0].index,'c']
    df['wick_trigger'] = 0


    col_loc = df.columns.get_loc(col)
    arr = df.values
    for i in range(lookback_threshold,arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        try:
            cur_max = max(arr[i - lookback_threshold : i ,col_loc])
        except:
            print('ERROR',i)
            cur_max = 100
        if arr[i,col_loc] > cur_max and sum(arr[i - 100 : i,-1]) == 0:
            arr[i,-1] = 1
    return pd.DataFrame(arr,columns = df.columns)
def consecutive_candles(df,num = 10,col = 'delta_vector'):
    print('CREATING CONSECUTIVE CANDLES TRIGGER...')
    df[col + '_consecutive_trigger'] = 0
    col_loc = df.columns.get_loc(col)
    arr = df.values
    last = -1
    counter = 0
    for i in range(arr.shape[0]):
        
        if i % 200000 == 0:
            print(i)
            
        if arr[i-1,col_loc] < 0 and arr[i,col_loc] < 0:
            counter += 1
        elif arr[i-1,col_loc] > 0 and arr[i,col_loc] > 0:
            counter += 1  
        else:
            counter = 0
            
        if counter == num:
            arr[i,-1] = 1
            counter = 0
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE OF TRIGGER DF:',df[df[col + '_consecutive_trigger'] == 1].shape)
    return df

def get_pearsons_corr(df,lookback = 10):
    print('GETTING PEARSONS CORR')
    df['ind'] = list(range(df.shape[0]))
    df['pearsons_corr'] = 0
    ind_col = df.columns.get_loc('ind')
    o_col = df.columns.get_loc('o')
    
    arr = df.values
    for i in range(arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        r, p = sp.stats.pearsonr(arr[(i - looback) : i,ind_col], arr[(i - looback) : i,o_col])
        arr[i,-1] = r
    return pd.DataFrame(arr,columns = df.columns)

def get_pearsons_corr(df,lookback = 10):
    print('GETTING PEARSONS CORR')
    df['ind'] = list(range(df.shape[0]))
    df['pearsons_corr'] = 0
    ind_col = df.columns.get_loc('ind')
    o_col = df.columns.get_loc('o')
    
    arr = df.values
    for i in range(lookback,arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        r, p = sp.stats.pearsonr(arr[(i - lookback) : i,ind_col], arr[(i - lookback) : i,o_col])
        arr[i,-1] = r
    return pd.DataFrame(arr,columns = df.columns)
def get_pearsons_corr2(df,lookback = 10):
    print('GETTING PEARSONS CORR')
    df['ind'] = list(range(df.shape[0]))
    df['pearsons_lookup'] = 0
    df['pearsons_corr2'] = 0
    ind_col = df.columns.get_loc('ind')
    o_col = df.columns.get_loc('o')
    c_col = df.columns.get_loc('c')
    new_col = df.columns.get_loc('pearsons_lookup')
    
    arr = df.values
    for i in range(lookback,arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        if i % 2 == 0:
            arr[i,new_col] = arr[i,o_col]
        else:
            arr[i,new_col] = arr[i,c_col]        
            
            
    for i in range(lookback,arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        r, p = sp.stats.pearsonr(arr[(i - lookback) : i,ind_col], arr[(i - lookback) : i,new_col])
        arr[i,-1] = r
    return pd.DataFrame(arr,columns = df.columns)
def convert_timestamp(df):
    time_col = df.columns.get_loc('time')
    arr = df.values 
    for i in range(arr.shape[0]):
        arr[i,time_col] = datetime.strptime(arr[i,time_col][:-4], '%Y-%m-%dT%H:%M:%S.%f')
    return pd.DataFrame(arr,columns = df.columns)
def get_best_fit(df,lookback = 10):

    def get_slope(y1,y2,total_x = 10):
        """y = mx + b"""
        return (y2 - y1) / total_x
    def get_best_fit_vals(y1,y2,total_x = 10):
        m = get_slope(y1,y2,total_x = total_x)
        lst = []
        for i in range(total_x):
            lst.append((m * i) + y1)
        return lst
    def compare_vals(lst,open_list,close_list,total_x):

        for i in range(total_x):
            if lst[i] <= max(open_list[i],close_list[i]) and lst[i] >= min(open_list[i],close_list[i]):
                pass
            else:
                return 0
        return 1
    print('GETTING BEST FIT INDICATOR')
    df['best_fit'] = 0
    o_col = df.columns.get_loc('o')
    c_col = df.columns.get_loc('c')
    new_col = df.columns.get_loc('best_fit')
    
    arr = df.values
    total_x = lookback 
    for i in range(lookback,arr.shape[0]):
        if i % 200000 == 0:
            print(i)

        y1 = np.mean([arr[i - total_x,o_col],arr[i - total_x,c_col]])
        y2 = np.mean([arr[i ,o_col],arr[i ,c_col]])
        
        temp_lst = get_best_fit_vals(y1,y2,total_x)
        arr[i,new_col] = compare_vals(lst = temp_lst,open_list = list(arr[i - total_x:i,o_col]),close_list = list(arr[i - total_x:i,c_col]),total_x = total_x)
            
    
    return pd.DataFrame(arr,columns = df.columns)   

def print_example(df,ind_list,ind = 30,delta = 240,trendline_lookback = 20):
    i = ind_list[ind]   
    print('row loc:',i,' ind:',ind,' delta:',delta)
    fig = go.Figure(data=go.Candlestick(x=df.iloc[i - delta:i + (delta*2),:]['time'],
                        open=df.iloc[i - delta:i + (delta*2),:]['o'],
                        high=df.iloc[i - delta:i + (delta*2),:]['h'],
                        low=df.iloc[i - delta:i + (delta*2),:]['l'],
                        close=df.iloc[i - delta:i + (delta*2),:]['c']))


    fig.add_vrect(x0=df['time'].iloc[i - 1], x1=df['time'].iloc[i+1], 
                  annotation_text="trigger point" , annotation_position="top left",
                  fillcolor="green", opacity=0.25, line_width=0)
    
    fig.add_shape(type='line',
                    x0=df.iloc[i - trendline_lookback,:]['time'],
                    y0=df.iloc[i - trendline_lookback,:]['o'],
                    x1=df.iloc[i,:]['time'],
                    y1=df.iloc[i ,:]['o'],
                    line=dict(color='Red',),
                    xref='x',
                    yref='y'
    )
    
    
    try:
        plot(fig)
    except:
        from plotly.offline import plot
        plot(fig)
def get_max_min_open_or_close(df):
    o_col = df.columns.get_loc('o')
    c_col = df.columns.get_loc('c')
    df['max_o_c'] = 0
    df['min_o_c'] = 0
    max_col = df.columns.get_loc('max_o_c')
    min_col = df.columns.get_loc('min_o_c')    
    arr = df.values
    for i in range(arr.shape[0]):
        arr[i,max_col] = max(arr[i,o_col],arr[i,c_col])
        arr[i,min_col] = min(arr[i,o_col],arr[i,c_col])
    return pd.DataFrame(arr,columns = df.columns)

def channel_indicator(df,lookback = 20):
    """Take the max of a bunch of consecutive candles and find the slope of the line
    Take the min of a bunch of consecutive candles and find the slope of the line
    
    """
    def get_slope(y1,y2,total_x = 10):
        """y = mx + b"""
        return (y2 - y1) / total_x
    def get_best_fit_vals(y1,y2,total_x = 10):
        m = get_slope(y1,y2,total_x = total_x)
        lst = []
        for i in range(total_x):
            lst.append((m * i) + y1)
        return lst
    def compare_vals(lst,check_list,total_x,check_type = 'max'):

        for i in range(total_x):
            if check_type == 'max':
                if lst[i] >= check_list[i]:
                    pass
                else:
                    return 0
            if check_type == 'min':
                if lst[i] <= check_list[i]:
                    pass
                else:
                    return 0
        return 1
    
    
    print('GETTING CHANNEL INDICATOR')
    df['channel_indicator'] = 0
    max_col = df.columns.get_loc('max_o_c')
    min_col = df.columns.get_loc('min_o_c')
    new_col = df.columns.get_loc('channel_indicator')
    df['spread_indicator'] = 0
    spread_col = df.columns.get_loc('spread_indicator')
    
    arr = df.values
    total_x = lookback 
    for i in range(lookback,arr.shape[0]):
        if i % 200000 == 0:
            print(i)

        y1 = arr[i - total_x,max_col]
        y2 = arr[i ,max_col]
        
        temp_lst1 = get_best_fit_vals(y1,y2,total_x)
        check_1 = compare_vals(lst = temp_lst1,
                                      check_list = list(arr[i - total_x:i,min_col]),
                                      total_x = total_x,
                                      check_type = 'max'
                                     )
        
        
        y1 = arr[i - total_x,min_col]
        y2 = arr[i ,min_col]
        
        temp_lst2 = get_best_fit_vals(y1,y2,total_x)
        check_2 = compare_vals(lst = temp_lst2,
                                      check_list = list(arr[i - total_x:i,max_col]),
                                      total_x = total_x,
                                      check_type = 'min'
                                     )   
        spread = np.array(temp_lst1) - np.array(temp_lst2)     
        arr[i,spread_col] = max(spread)
        
        
        if check_1 == 1 and check_2 == 1:
            arr[i,new_col] = 1
    
    return pd.DataFrame(arr,columns = df.columns)  


def get_support(df,lookup_range = 60,stop_range = 20000,lookup_range2 = 200):
    print('GETTING SUPPORT INDICATOR')
    s = time.time()
    c_col = df.columns.get_loc('c')
    l_col = df.columns.get_loc('l')
    df['support_lookup'] = 0
    df['support_indicator'] = 0
    lookup_col = df.columns.get_loc('support_lookup')
    new_col = df.columns.get_loc('support_indicator')
    arr = df.values
    
    for i in range(arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        try:
            if arr[i,c_col] == min(arr[i - lookup_range : i + lookup_range,c_col]):
                lookup_ind = i
                val = arr[i,c_col]
                lookup_check = 0
                for j in range(i + lookup_range2,i + stop_range):
                    if arr[j,l_col] <= val and lookup_check == 0:
                        arr[j,new_col] = 1
                        arr[j,lookup_col] = lookup_ind
                        lookup_check = 1
                        break
        except:
            pass
        

    e = time.time()
    print('TOTAL FUNCTION TIME:',(e-s)/60,' MINUTES')
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['support_indicator'] == 1].shape)
    return df

def ema(df,num = 14):
    print('GETTING EMA INDICATOR FOR:',num)
    close_col = df.columns.get_loc('c')
    df['ema_' + str(num)] = df['c']
    arr = df.values
    mult = 2/ (num + 1)
    for i in range(num,arr.shape[0]):
        sma = sum(arr[i - num + 1: i + 1,close_col]) / num
        arr[i,-1] = ((arr[i,close_col] - arr[i - 1,-1]) * mult) + arr[i - 1,-1]
    return pd.DataFrame(arr,columns = df.columns)

file = 'XAU_USD_M1_2019-01-01_2022-01-31.csv'
#file = 'GBP_JPY_M1_2019-01-01_2022-01-31.csv'
#file = 'XAU_USD_S5/2020-08-01_2020-09-01.csv'
#file = 'GBP_JPY_S5/2020-08-01_2020-09-01.csv'

file = 'EUR_USD_M1_2019-01-01_2022-01-31.csv'
file = 'GBP_JPY_M1_2019-01-01_2022-01-31.csv'
file = 'EUR_USD_M1_2019-01-01_2022-01-31.csv'
file = 'GBP_JPY_M1_2019-01-01_2022-01-31.csv'

file = 'EUR_USD_M1_2019-01-01_2022-01-31.csv'

df = pd.read_csv(file)
df = convert_timestamp(df)
df = get_max_min_open_or_close(df)
print(df.shape)
df['o'] = df['o'].astype(float)
df['h'] = df['h'].astype(float)
df['l'] = df['l'].astype(float)
df['c'] = df['c'].astype(float)

df = get_support(df,lookup_range = 1500,stop_range = 20000,lookup_range2 = 200)

df = ema(df,num = 14 * 15)

df = ema(df,num = 50 * 15)
df = ema(df,num = 200 * 15)
#df = ema(df,num = 500 * 15)

df.head()